<a href="https://colab.research.google.com/github/jeffheaton/t81_558_deep_learning/blob/master/t81_558_class_07_2_train_gan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 01. Clone the repo and install `ninja`

Paperspace has a built-in PyTorch version already installed depending on your configuration.

Make sure to prepare a notebook with torch>=1.9.0.

In [ ]:
!git clone https://github.com/NVlabs/stylegan3.git
!pip install ninja

### 02. Create folders for training

Create a folder structure in your Paperspace gradient notebook file system:

```
├── stylegan
│   ├── images
│   │   ├── <Your dataset folder containing images with dimensions of power of 2 (ex. - 64x64, 128x128, 1024x1024)>
│   ├── datasets
│   ├── experiments
```

You can use [ImageMagick's](https://imagemagick.org/) batch image processor script `mogrify` locally before uploading to make sure all your images have a power-of-two dimension. You can also use PIL to do this as well.

Take note of your raw dataset folder name and put it in the `RAW_DATASET_FOLDER_NAME` variable below.


In [2]:
import os

# Change this to your dataset folder name
RAW_DATASET_FOLDER_NAME = "faces" 

STYLEGAN_ROOT_FOLDER = os.path.join(
    os.sep, "notebooks", "stylegan"
)
RAW_DATASET_PATH = os.path.join(STYLEGAN_ROOT_FOLDER, "images", RAW_DATASET_FOLDER_NAME)
ZIP_DATASET_PATH = os.path.join(STYLEGAN_ROOT_FOLDER, "datasets", RAW_DATASET_FOLDER_NAME)
EXPERIMENTS_PATH = os.path.join(STYLEGAN_ROOT_FOLDER, "experiments")

STYLEGAN_PATH = os.path.join(os.sep, "notebooks", "stylegan3")
DATASET_TOOL_PATH = os.path.join(STYLEGAN_PATH, "dataset_tool.py")
TRAINING_SCRIPT_PATH = os.path.join(STYLEGAN_PATH, "train.py")

In [ ]:
dataset_creation_cmd = f"python {DATASET_TOOL_PATH} --source={RAW_DATASET_PATH} --dest={ZIP_DATASET_PATH}"
!{dataset_creation_cmd}

In [ ]:
# Verifies your dataset for inconsistent sizes and colour formats
# Script created by @jeffheaton

from os import listdir
from os.path import isfile, join
import os
from PIL import Image
from tqdm import tqdm


files = [f for f in listdir(RAW_DATASET_PATH) if isfile(join(RAW_DATASET_PATH, f))]

base_size = None
for file in tqdm(files):
    file2 = os.path.join(RAW_DATASET_PATH, file)
    img = Image.open(file2)
    sz = img.size
    if base_size and sz != base_size:
        print(f"Inconsistant size: {file2}")
    elif img.mode != "RGB":
        print(f"Inconsistant color format: {file2}")
    else:
        base_size = sz

### 03. Training

You can play around with different configurations here. A few descriptions:

- `CFG` - you can pick `stylegan3-t` (only designed for translation equivariance) or `stylegan3-r` (features high-quality, though not visually perfect rotation equivariance). More details in the [official NVIDIA StyleGAN3 documentation](https://nvlabs.github.io/stylegan3/)
- `BATCH_SIZE` - the number of samples that will be passed through to the network at one time
- `BATCH_GPU` - per-GPU batch size
- `GAMMA` - R1 regularization weight
- `SNAP` - number of epochs before saving a new checkpoint in the `experiments` folder on Google Drive (for Google Colab Free, it is recommended to keep this low, say `5` or `10`)

You can refer to the [official documentation's recommended configurations](https://github.com/NVlabs/stylegan3/blob/main/docs/configs.md#recommended-configurations) to see more config parameters.

In [ ]:
import os

# Modify these to suit your needs
CFG = "stylegan3-t"
BATCH_SIZE = 32
BATCH_GPU = 16
GAMMA = 0.5
SNAP = 10

# Build the command and run it
cmd = f"python stylegan3/train.py --outdir={EXPERIMENTS_PATH} --data={ZIP_DATASET_PATH} --cfg={CFG} --gpus=1 --workers=1 --batch={BATCH_SIZE} --batch-gpu={BATCH_GPU} --gamma={GAMMA} --snap={SNAP}"
!{cmd}

### 04. Resuming training

Your Paperspace instance will automatically be turned off after some time depending on your auto-shutdown setting (Max 6 hours for free instances).
You can resume training with the code below.

Check your experiments folder to check the `CHECKPOINT_FOLDER` and `NETWORK` name in the `experiments` folder.
An example of this is:

```
├── stylegan
│   ├── images
│   ├── dataset
│   ├── experiments
│   │   ├── 00002-dogs_images-auto1-resumecustom --> This is the `CHECKPOINT_FOLDER`
│   │   │   ├── network-snapshot-000160.pkl --> This is the `NETWORK`
```

You can use the `NETWORK` with the latest epoch to continue.
The confgurations are the same as the ones in *04. Training*.

In [ ]:
# Resume Training

import os

# Modify these to suit your needs
CHECKPOINT_FOLDER = "<CHANGE_ME>"
NETWORK = "<CHANGE_ME>"

RESUME = os.path.join(EXPERIMENTS_PATH, CHECKPOINT_FOLDER, NETWORK)
CFG = "stylegan3-t"
BATCH_SIZE = 32
BATCH_GPU = 16
GAMMA = 0.5
SNAP = 10

# Build the command and run it
cmd = f"python stylegan3/train.py --resume={RESUME} --outdir={EXPERIMENTS_PATH} --data={ZIP_DATASET_PATH} --cfg={CFG} --gpus=1 --workers=1 --batch={BATCH_SIZE} --batch-gpu={BATCH_GPU} --gamma={GAMMA} --snap={SNAP}"
!{cmd}